<a href="https://colab.research.google.com/github/adilabduakhanov/Developing_Neuro-Employees_on_GPT/blob/main/%D0%A1%D1%82%D1%80%D1%83%D0%BA%D1%82%D1%83%D1%80%D0%B8%D1%80%D0%BE%D0%B2%D0%B0%D0%BD%D0%BD%D1%8B%D0%B5_%D0%BE%D1%82%D0%B2%D0%B5%D1%82%D1%8B_GPT_%D0%B8_functional_calling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip -q install -U openai

import os
from openai import OpenAI
from google.colab import userdata

# В Colab удобно так:
# 1. Сохраните ваш API-ключ в Colab Secrets под именем 'OPENAI_API_KEY'.
# 2. Раскомментируйте следующую строку, чтобы использовать ключ из Secrets:
os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')

# Если вы хотите вставить ключ напрямую (НЕ РЕКОМЕНДУЕТСЯ для продакшн):
# os.environ["OPENAI_API_KEY"] = "sk-..."   # <-- вставь ключ сюда

client = OpenAI()

In [ ]:
import json
from pathlib import Path
from datetime import date

DB_PATH = Path("tasks.json")

# Формат БД:
# {
#   "YYYY-MM-DD": [
#       {"title": "...", "time": "HH:MM" or None, "category": "...", "raw_text": "..."}
#   ],
#   ...
# }

def load_db():
    if DB_PATH.exists():
        return json.loads(DB_PATH.read_text(encoding="utf-8"))
    return {}

def save_db(db):
    DB_PATH.write_text(json.dumps(db, ensure_ascii=False, indent=2), encoding="utf-8")

db = load_db()
print("Loaded dates:", list(db.keys())[:5])


Loaded dates: []


In [ ]:
from typing import Optional, Dict, Any, List

def add_task(date_iso: str, title: str, time_hhmm: Optional[str] = None,
             category: str = "прочее", raw_text: str = "") -> Dict[str, Any]:
    """Добавить задачу в память."""
    if date_iso not in db:
        db[date_iso] = []
    db[date_iso].append({
        "title": title.strip(),
        "time": time_hhmm,
        "category": category.strip(),
        "raw_text": raw_text.strip()
    })
    save_db(db)
    return {
        "ok": True,
        "created": {
            "date": date_iso,
            "time": time_hhmm,
            "title": title,
            "category": category
        }
    }

def list_tasks(date_iso: str) -> Dict[str, Any]:
    """Показать задачи на дату."""
    tasks = db.get(date_iso, [])
    # сортировка: сначала с временем, потом без
    def sort_key(t):
        return (t["time"] is None, t["time"] or "99:99")
    tasks_sorted = sorted(tasks, key=sort_key)
    return {"date": date_iso, "tasks": tasks_sorted, "count": len(tasks_sorted)}


In [ ]:
TOOLS = [
    {
        "type": "function",
        "function": {
            "name": "add_task",
            "description": "Записать задачу/дело пользователя на конкретную дату (и время, если указано).",
            "strict": True,
            "parameters": {
                "type": "object",
                "additionalProperties": False,
                "properties": {
                    "date_iso": {
                        "type": "string",
                        "description": "Дата в формате YYYY-MM-DD"
                    },
                    "title": {
                        "type": "string",
                        "description": "Короткое название дела (без даты/времени)"
                    },
                    "time_hhmm": {
                        "type": ["string", "null"],
                        "description": "Время в формате HH:MM или null, если времени нет"
                    },
                    "category": {
                        "type": "string",
                        "description": "Категория: встреча/звонок/задача/другое и т.п."
                    },
                    "raw_text": {
                        "type": "string",
                        "description": "Оригинальный текст пользователя (как пришёл)"
                    }
                },
                "required": ["date_iso", "title", "time_hhmm", "category", "raw_text"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "list_tasks",
            "description": "Вывести список дел на заданную дату.",
            "strict": True,
            "parameters": {
                "type": "object",
                "additionalProperties": False,
                "properties": {
                    "date_iso": {
                        "type": "string",
                        "description": "Дата в формате YYYY-MM-DD"
                    }
                },
                "required": ["date_iso"]
            }
        }
    }
]


In [ ]:
from datetime import datetime

SYSTEM = f"""
Ты — ассистент-планировщик задач.
Твоя задача:
1) Если пользователь сообщает новое дело (в понедельник..., завтра..., 14:00..., на 2025-07-14...), ты ДОЛЖЕН вызвать tool add_task.
2) Если пользователь просит показать дела на день (что у меня завтра? список на понедельник? дела на 2025-07-14?), ты ДОЛЖЕН вызвать tool list_tasks.
3) Если пользователь пишет что-то не про планирование — задай уточнение, но без tool.

Важно:
- Сегодняшняя дата: {date.today().isoformat()}
- Преобразуй "завтра/послезавтра/в понедельник/через 3 дня" в конкретную дату YYYY-MM-DD.
- Время нормализуй в HH:MM (например "в 10" -> "10:00").
- title делай коротким ("встреча по проекту"), без даты и времени.
- category: встреча/звонок/задача/прочее.
- В итоговом ответе после tool используй формат как в примере:
  '... ' создано на YYYY-MM-DD в HH:MM (категория: ...).
  или для списка: "Дела на YYYY-MM-DD: ..." (с нумерацией).
""".strip()

def run_planner(user_text: str, model: str = "gpt-4o"):
    messages = [
        {"role": "system", "content": SYSTEM},
        {"role": "user", "content": user_text}
    ]

    # 1) первый вызов — модель решает, нужно ли вызывать tool
    resp = client.chat.completions.create(
        model=model,
        messages=messages,
        tools=TOOLS,
        tool_choice="auto",
    )
    msg = resp.choices[0].message

    # Если tool_calls нет — просто текст
    if not getattr(msg, "tool_calls", None):
        return msg.content

    # 2) выполняем tools
    messages.append({
        "role": "assistant",
        "content": msg.content,
        "tool_calls": [tc.model_dump() for tc in msg.tool_calls]
    })

    for tc in msg.tool_calls:
        name = tc.function.name
        args = json.loads(tc.function.arguments)

        if name == "add_task":
            result = add_task(**args)
        elif name == "list_tasks":
            result = list_tasks(**args)
        else:
            result = {"ok": False, "error": f"Unknown tool: {name}"}

        messages.append({
            "role": "tool",
            "tool_call_id": tc.id,
            "content": json.dumps(result, ensure_ascii=False)
        })

    # 3) второй вызов — модель формирует финальный ответ пользователю,
    # опираясь на результат tool
    resp2 = client.chat.completions.create(
        model=model,
        messages=messages,
    )
    return resp2.choices[0].message.content


In [ ]:
print(run_planner("в понедельник встреча по проекту в 14"))
print(run_planner("завтра созвон с заказчиком в 10"))
print(run_planner("что у меня завтра?"))


'встреча по проекту' создано на 2025-12-29 в 14:00 (категория: встреча).
'созвон с заказчиком' создано на 2025-12-24 в 10:00 (категория: звонок).
Дела на 2025-12-24:
1. 10:00 - созвон с заказчиком (категория: звонок).


In [ ]:
while True:
    user = input("Вы: ").strip()
    if user.lower() in {"выход", "exit", "quit"}:
        break
    answer = run_planner(user)
    print("Планировщик:", answer)


Вы: Планировщик
Планировщик: Могу вам помочь с планированием задач. Вы хотите добавить новое дело или просмотреть список дел?


KeyboardInterrupt: Interrupted by user